In [115]:
import torch
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np

fc = 1
#na = np.empty((1, 51))
na = []

In [116]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [117]:
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (567, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 567, 960])
    if torch.cuda.is_available():
        image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 567, 960])
    with torch.no_grad():
        output, _ = model(image)
    return output, image

In [122]:
def draw_keypoints(output, image):
    global fc
    output = non_max_suppression_kpt(output, 
                                     0.1, # Confidence Threshold
                                     0.2, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
        #print(f'Frame Number: {fc}; Data Size: {output.shape}')
    singleSkel = []
    try:
        t = output[0] #retrieves only first skeleton data
        t = t[-51:] #retrieves last 51 elements
        #t = t[::3] cuts every third element (confidence level)
        t = [x for i, x in enumerate(t) if (i+1)%3 != 0]
        na.append(t)
        """
        for i in range(17):
            coords = [t[2*i], t[2*i+1]]
            singleSkel.append(coords)
        na.append(singleSkel)
    except:
        for i in range(17):
            coords = [np.nan, np.nan]
            singleSkel.append(coords)
        na.append(singleSkel)
        #na.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,])
        #na.append([0])
        """
        
    except:
        na.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,])
        #na.append([null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null,null, null])
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    
    
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
    
    
        
    return nimg

In [119]:
"""
def draw_keypoints(output, image):
    global fc
    output = non_max_suppression_kpt(output, 
                                     0.08, # Confidence Threshold
                                     0.25, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
        print(f'Frame Number: {fc}; Data Size: {output.shape}')
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    
    
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
    
    
        
    return nimg
    """

"\ndef draw_keypoints(output, image):\n    global fc\n    output = non_max_suppression_kpt(output, \n                                     0.08, # Confidence Threshold\n                                     0.25, # IoU Threshold\n                                     nc=model.yaml['nc'], # Number of Classes\n                                     nkpt=model.yaml['nkpt'], # Number of Keypoints\n                                     kpt_label=True)\n    with torch.no_grad():\n        output = output_to_keypoint(output)\n        print(f'Frame Number: {fc}; Data Size: {output.shape}')\n    nimg = image[0].permute(1, 2, 0) * 255\n    nimg = nimg.cpu().numpy().astype(np.uint8)\n    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)\n\n    \n    \n    for idx in range(output.shape[0]):\n        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)\n    \n    \n        \n    return nimg\n    "

In [120]:
def pose_estimation_video(filename):
    global fc
    cap = cv2.VideoCapture(filename)
    # VideoWriter for saving the video
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter('tempOutput.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
    while cap.isOpened():
        (ret, frame) = cap.read()
        if ret == True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output, frame = run_inference(frame)
            frame = draw_keypoints(output, frame)
            fc += 1 #
            frame = cv2.resize(frame, (int(cap.get(3)), int(cap.get(4))))
            frame = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            out.write(frame)
            cv2.imshow('Pose estimation', frame)
        else:
            break

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [128]:
video = "C:/Users/jonso/OneDrive/Desktop/IMG_5498.MOV"
video = "C:/Users/jonso/OneDrive/Desktop/swim_rio_1.mp4"
pose_estimation_video(video)
#""C:\Users\jonso\OneDrive\Desktop\Smooth swimming step by step.mp4"
#"C:\Users\jonso\OneDrive\Desktop\swim_rio_1.mp4"
#"C:\Users\jonso\OneDrive\Desktop\IMG_5497.MOV"

In [127]:
skelData = np.array(na)
skelData = skelData.reshape(skelData.shape[0], 17, 2)
print(skelData.shape)
#print(na)
print(skelData)

(853, 17, 2)
[[[        699       321.5]
  [        700      319.75]
  [        697         322]
  ...
  [      786.5      325.75]
  [      792.5       307.5]
  [        785       307.5]]

 [[      698.5      300.75]
  [        699      298.75]
  [        696         301]
  ...
  [        790         329]
  [        793         307]
  [        796       307.5]]

 [[        712         307]
  [        714      306.75]
  [        712      307.25]
  ...
  [      751.5       289.5]
  [        790       307.5]
  [        787       300.5]]

 ...

 [[        438      310.25]
  [      435.5      311.75]
  [     440.25         310]
  ...
  [        606      352.75]
  [        700      356.75]
  [        726         364]]

 [[     379.75      299.75]
  [     376.25         299]
  [      381.5      299.75]
  ...
  [      594.5       355.5]
  [      697.5         355]
  [        707      361.75]]

 [[        716      327.75]
  [        722       328.5]
  [      712.5         319]
  ...
  [     471

In [ ]:
Frame Number: 84; Data Size: (1, 58)
        
        batch_id, class_id, x, y, w, h, conf
        
[[          0           0      724.97      320.72      201.62      94.438     0.55844         
  0 699       321.5     0.24487         
  1 700      319.75     0.17566         
  2 697         322    0.088501       
  3 708.5      316.75     0.28247       
  4 700.5      320.75     0.13757         
  5 729         301     0.81641         
  6 700        314.5      0.8335       
  7 753.5      290.75     0.87598       
  8 677.5      307.25     0.87158         
  9 756        316.25     0.85107       
  10 650.5      322        0.85352       
  11 756.5      305        0.88818       
  12 744.5      313.5      0.89648         
  13 794        314.75     0.7168       
  14 786.5      325.75     0.74707
  15 792.5      307.5      0.67725         
  16 785        307.5      0.70459]]



In [ ]:
g = np.load("C:/Users/jonso/OneDrive/Desktop/Trinh Project/x_train.npy")
print(g.shape)